In [1]:
import selenium
import pandas as pd
import time
import  requests
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings('ignore')

In [6]:
driver=webdriver.Chrome(r"chromedriver.exe")

## Opening the wesite

In [7]:
url = "https://www.flipkart.com/"
driver.get(url)
driver.implicitly_wait(3)

try:
    cancel_button = driver.find_element_by_class_name('_2doB4z')
    cancel_button.click()
    
except:
    print('no registartion page pop up!!')

In [8]:
srch_items = ['laptops', 'Smart Phones','smart watches', 'AC']

## Lets store url of each product in a list
**we have stored the url in a list urls, later we will scrap each url for review and ratings**

In [9]:
urls = []

for item in srch_items:
    srch_bar= driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input") # search bar
#     time.sleep(3)
    srch_bar.clear() # clearing the search bar
    srch_bar.send_keys(item) # typing item from list into the search bar
    click_on_serach = driver.find_element_by_xpath("//button[@class='L0Z3Pu']") # slecting the search button
    click_on_serach.click() # clicking the search button
    search_result = driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a") # selenium object for searched result
    pages = [] 
    for page in search_result:  ## looping the all the pages which we need to scrap
        pages.append(page.get_attribute('href'))

    for page in pages[0:4]: # lets scarp 4 pages of the search result.
        driver.get(page)
        driver.implicitly_wait(3)
        product = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
        for url in product:
            urls.append(url.get_attribute('href'))


In [10]:
len(urls) ## url for products. later we will scrap the review 

360

#### Defining a function for review scraping

In [1]:
titles = []
review_text = []
ratings = []

#### defining a function for the review scraping

In [12]:
def scrap():
    
    # Scraping Ratings
    for rating in driver.find_elements_by_xpath("//p[@class='_2-N8zT']/../div"):
        ratings.append(rating.text)
        
    # Scraping Title
    
    for title in driver.find_elements_by_xpath("//p[@class='_2-N8zT']"):
        titles.append(title.text)
        
    ## Scraping the review
    for review in driver.find_elements_by_xpath("//div[@class='t-ZTKy']"):
        review_text.append(review.text)

In [14]:
for url in urls:

    driver.get(url)
    driver.implicitly_wait(5)
    # driver.execute_script("window.scrollBy(0,4000)")
    try:
        # opening the link for the view review page
        lnk =driver.find_element_by_xpath("//div[@class='_2c2kV-']/following::a").get_attribute('href')
        driver.get(lnk) # opening the review page
        driver.implicitly_wait(5)
    except NoSuchElementException:
        pass
    scrap()

    try:
        # Accessing the multiple page of the riview section
        n_pages = driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a")
        n_page = []
        for page in n_page[:-1]: # -1 is used for avoid the repetition due to next button
            n_page.append(page.get_attribute('href'))
        for i in page[0:10]: # this will acces 10 pages of the review section
            driver.get(i)
            driver.implicitly_wait(5)
            scrap()
    except: continue

In [15]:
len(ratings), len(review_text), len(titles) ## number of reviews scrapped

(3228, 3228, 3228)

### Lets convert the data into pandas dataframe

In [17]:
data = list(zip(titles,review_text,ratings))
df = pd.DataFrame(data, columns = ["Review_title","Reiew_text","Ratings"])

In [19]:
df.head(10)

,Review_title,Reiew_text,Ratings
0,Simply awesome,Superb design value for money,5
1,Best in the market!,"Excellent, fast, Efficient, If you are looking...",5
2,Wonderful,Love one,5
3,Great product,Nice product and same as seen as the product.t...,5
4,Pretty good,As of now okay doing good performance for the ...,4
5,Mind-blowing purchase,Nice product,5
6,Terrible product,When i open a lid andvibam not pressing the po...,1
7,Slightly disappointed,Iam shutdown my laptop but when to i open lapt...,2
8,Good quality product,Good Product,4
9,Great product,Great,5


In [21]:
df.to_csv(r'C:\Users\katiy\Desktop\flipkart_review.csv') ## Saving the data in csv format